In [75]:
from qdrant_client.http import models as rest
import pandas as pd

In [76]:
activity_df = pd.read_json('raw_data/MyActivity.json')

In [77]:
# Remove rows containing "Visited" and "Viewed" in the title
activity_df = activity_df[~activity_df['title'].str.contains('Visited')]
activity_df = activity_df[~activity_df['title'].str.contains('Viewed')]

# Remove "Searched for " in the title
activity_df['title'] = activity_df['title'].str.replace('Searched for ', '')
activity_df = activity_df.sort_values(by='time')
activity_df.reset_index(inplace=True, drop=True)

activity_df.to_csv('raw_data/activity.csv')

In [78]:
df = pd.read_csv('raw_data/embeddings.csv')

In [79]:
from qdrant_client import QdrantClient

client = QdrantClient(
    url="https://fee969cc-0326-4cf7-85dc-9d710984e8dd.us-east4-0.gcp.cloud.qdrant.io", 
    api_key="IWftJu7oEPz24XSqNfhLhwGt7urtvDpmDauP0NUXT79xV_xUpvIHdA",
)

In [80]:
# Create collection
vector_size = 1024
client.recreate_collection(
    collection_name="Searches",
    vectors_config={
        "search": rest.VectorParams(
            distance=rest.Distance.COSINE,
            size=vector_size,
        ),
    }
)

True

In [81]:
# Store all the vectors

def upsert_vectors(df):
    points = [
        rest.PointStruct(
            id=k,
            payload={
                "search_string": activity_df['title'][k]
            },
            vector={
                "search": v.values.tolist(),
            },
        ) for k, v in df.iterrows()
    ]
    
    client.upsert(
        collection_name="Searches",
        points=points
    )

In [82]:
# Slice the df in chunks of 1000 rows and call upsert_vectors for each chunk 
from math import ceil

from tqdm import tqdm

chunk = 100
for i in tqdm(range(0, ceil(len(df)/chunk))):
    upsert_vectors(df.iloc[i*chunk:min((i+1)*chunk, len(df))])

  0%|          | 0/688 [00:00<?, ?it/s]

100%|██████████| 688/688 [12:56<00:00,  1.13s/it]


In [83]:
# Check the collection size to make sure all the points have been stored
client.count(collection_name="Searches")

CountResult(count=68763)